# Task(10) AI Report Automation Chatbot

## 📋 프로젝트 개요

OpenAI API와 웹 검색 기능을 활용하여 사용자가 입력한 주제에 대해 자동으로 구조화된 리포트를 생성하는 대화형 챗봇 시스템입니다.

### 주요 기능
- **주제 명확성 판단**: 입력된 주제가 구체적인지 모호한지 자동 판단
- **Clarify 단계**: 모호한 주제에 대한 추가 질문으로 구체화 유도
- **웹 검색 기반 리포트**: 신뢰할 만한 출처를 통한 최신 정보 수집
- **구조화된 리포트**: Keywords, Sources, Summary, Body 4개 섹션으로 구성

### 기술 스택
- **OpenAI API**: gpt-4o (대화 처리), gpt-4o-search-preview (웹 검색)
- **Python**: 주요 프로그래밍 언어
- **python-dotenv**: 환경 변수 관리


In [1]:
# 환경 설정 및 라이브러리 import
import os
import json
import time
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
from dotenv import load_dotenv
from openai import OpenAI

# 환경 변수 로드
load_dotenv()

# OpenAI API 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 전역 설정
CONVERSATION_HISTORY = []
MAX_CLARIFY_ATTEMPTS = 2
MAX_HISTORY_LENGTH = 20

print("✅ 환경 설정 완료")
print(f"OpenAI API 키 설정: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")


✅ 환경 설정 완료
OpenAI API 키 설정: ✅


In [2]:
# 1. 주제 명확성 판단 함수
def is_topic_vague(topic: str) -> bool:
    """
    주제가 모호한지 판단하는 함수
    
    Args:
        topic: 사용자가 입력한 주제
        
    Returns:
        bool: True면 모호함, False면 명확함
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": """당신은 주제의 명확성을 판단하는 전문가입니다.
                    
다음 기준으로 주제가 모호한지 판단하세요:
1. 범위가 지나치게 포괄적인가? (예: "AI", "경제", "환경")
2. 구체적인 검색이 어려운가?
3. 여러 해석이 가능한가?

명확한 주제 예시:
- "AI가 교육에 미치는 영향"
- "2024년 한국 부동산 시장 전망"
- "코로나19가 온라인 쇼핑에 미친 영향"

모호한 주제 예시:
- "AI"
- "경제"
- "환경"
- "기술"

JSON 형태로 응답하세요: {"is_vague": true/false, "reason": "판단 이유"}"""
                },
                {
                    "role": "user",
                    "content": f"다음 주제의 명확성을 판단해주세요: '{topic}'"
                }
            ],
            temperature=0.3
        )
        
        result = json.loads(response.choices[0].message.content)
        return result["is_vague"]
        
    except Exception as e:
        print(f"❌ 주제 명확성 판단 중 오류: {e}")
        # 오류 시 안전하게 모호하다고 판단
        return True

# 테스트
test_topics = [
    "AI",  # 모호함
    "AI가 교육에 미치는 영향",  # 명확함
    "경제",  # 모호함
    "2024년 한국 부동산 시장 전망"  # 명확함
]

print("🧪 주제 명확성 판단 테스트:")
for topic in test_topics:
    is_vague = is_topic_vague(topic)
    status = "모호함" if is_vague else "명확함"
    print(f"  '{topic}' → {status}")


🧪 주제 명확성 판단 테스트:
  'AI' → 모호함
  'AI가 교육에 미치는 영향' → 명확함
  '경제' → 모호함
  '2024년 한국 부동산 시장 전망' → 명확함


In [3]:
# 2. Clarify 단계 구현
def clarify_question(topic: str, attempt: int = 1) -> str:
    """
    모호한 주제에 대한 추가 질문을 생성하는 함수
    
    Args:
        topic: 사용자가 입력한 주제
        attempt: 현재 시도 횟수 (1~2)
        
    Returns:
        str: 구체화를 유도하는 질문
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": f"""당신은 사용자 친화적인 AI 어시스턴트입니다.
                    
사용자가 입력한 주제가 모호하므로, 구체화를 유도하는 질문을 생성해야 합니다.

규칙:
1. 친근하고 도움이 되는 톤 사용
2. 구체적인 선택지를 제시 (2-3개)
3. 간결하고 명확한 질문
4. {MAX_CLARIFY_ATTEMPTS}회 이내로 제한

예시:
- "AI" → "AI에 대해 어떤 분야가 궁금하신가요? 교육, 의료, 자율주행, 창작 중에서 선택해주세요."
- "경제" → "경제 관련해서 어떤 주제가 관심 있으신가요? 부동산, 주식, 인플레이션, 일자리 중에서 말씀해주세요."

한 문장으로 간결하게 질문하세요."""
                },
                {
                    "role": "user",
                    "content": f"주제: '{topic}' (시도 {attempt}/{MAX_CLARIFY_ATTEMPTS})"
                }
            ],
            temperature=0.7
        )
        
        return response.choices[0].message.content.strip()
        
    except Exception as e:
        print(f"❌ Clarify 질문 생성 중 오류: {e}")
        return "주제를 좀 더 구체적으로 말씀해 주실 수 있을까요?"

# 테스트
test_vague_topics = ["AI", "경제", "환경", "기술"]

print("🧪 Clarify 질문 생성 테스트:")
for topic in test_vague_topics:
    question = clarify_question(topic)
    print(f"  '{topic}' → {question}")
    print()


🧪 Clarify 질문 생성 테스트:
  'AI' → AI에 대해 어떤 분야가 궁금하신가요? 교육, 의료, 자율주행, 창작 중에서 선택해주세요.

  '경제' → 경제 관련해서 어떤 주제가 관심 있으신가요? 부동산, 주식, 인플레이션, 일자리 중에서 말씀해주세요.

  '환경' → 환경과 관련해서 어떤 부분이 궁금하신가요? 기후 변화, 재활용, 에너지 절약 중에서 선택해주세요.

  '기술' → 기술에 대해 어떤 부분이 궁금하신가요? 인공지능, 가상현실, 블록체인 중에서 말씀해 주세요.



In [4]:
# 3. 리포트 생성 함수
def generate_report(topic: str):
    """
    웹 검색을 통해 구조화된 리포트를 생성하는 함수
    
    Args:
        topic: 사용자가 입력한 주제
        
    Returns:
        Dict: Keywords, Sources, Summary, Body를 포함한 리포트
    """
    try:
        print(f"🔍 '{topic}' 주제로 웹 검색 중...")
        
        response = client.chat.completions.create(
            model="gpt-4o-search-preview",
            messages=[
                {
                    "role": "system",
                    "content": """당신은 전문 리서치 어시스턴트입니다.
                    
다음 4개 섹션으로 구조화된 리포트를 생성하세요:

1. Keywords: 실제 사용한 주요 검색어 목록 (5-8개)
2. Sources: 신뢰할 만한 웹사이트 URL 목록 (3-5개)
3. Summary: 핵심 내용의 압축 서술 (5-7문장)
4. Body: 서론-본론-결론 구조의 분석 리포트 (1000-1500자)

요구사항:
- 신뢰할 만한 출처 우선 (정부기관, 학술기관, 언론사)
- 최신 정보 우선 (QDF 개념 적용)
- 객관적이고 균형잡힌 시각
- 한국어로 작성
- 구체적인 데이터와 사례 포함

JSON 형태로 응답하세요:
{
    "keywords": ["검색어1", "검색어2", ...],
    "sources": ["URL1", "URL2", ...],
    "summary": "요약 내용",
    "body": "본문 내용"
}"""
                },
                {
                    "role": "user",
                    "content": f"다음 주제에 대한 상세한 리포트를 작성해주세요: '{topic}'"
                }
            ]
        )
        
        # JSON 파싱
        content = response.choices[0].message.content
        try:
            # JSON 부분만 추출 (```json ... ``` 형태일 수 있음)
            if "```json" in content:
                json_start = content.find("```json") + 7
                json_end = content.find("```", json_start)
                json_content = content[json_start:json_end].strip()
            else:
                json_content = content.strip()
            
            report = json.loads(json_content)
            
            # 필수 필드 검증
            required_fields = ["keywords", "sources", "summary", "body"]
            for field in required_fields:
                if field not in report:
                    report[field] = f"[{field} 정보를 찾을 수 없습니다]"
            
            return report
            
        except json.JSONDecodeError:
            # JSON 파싱 실패 시 기본 구조로 반환
            return {
                "keywords": [topic],
                "sources": ["검색 결과를 파싱할 수 없습니다"],
                "summary": "검색 결과를 분석하는 중 오류가 발생했습니다.",
                "body": content
            }
        
    except Exception as e:
        print(f"❌ 리포트 생성 중 오류: {e}")
        return {
            "keywords": [topic],
            "sources": ["검색 실패"],
            "summary": "현재 해당 주제에 대한 신뢰할 만한 자료를 찾지 못했습니다.",
            "body": "검색 과정에서 오류가 발생했습니다. 다른 주제를 시도하시겠습니까?"
        }

def format_report(report, topic):
    """
    리포트를 보기 좋게 포맷팅하는 함수
    
    Args:
        report: generate_report()에서 반환된 리포트 딕셔너리
        topic: 주제
        
    Returns:
        str: 포맷팅된 리포트 문자열
    """
    formatted = f"# Report: {topic}\n\n"
    
    # Keywords
    formatted += "## 1. Keywords\n"
    formatted += ", ".join(report.get("keywords", [])) + "\n\n"
    
    # Sources
    formatted += "## 2. Sources\n"
    for i, source in enumerate(report.get("sources", []), 1):
        formatted += f"- {source}\n"
    formatted += "\n"
    
    # Summary
    formatted += "## 3. Summary\n"
    formatted += report.get("summary", "") + "\n\n"
    
    # Body
    formatted += "## 4. Body\n"
    formatted += report.get("body", "") + "\n"
    
    return formatted

# 테스트
test_topic = "AI가 교육에 미치는 영향"
print(f"🧪 리포트 생성 테스트: '{test_topic}'")
report = generate_report(test_topic)
formatted_report = format_report(report, test_topic)
print(formatted_report)


🧪 리포트 생성 테스트: 'AI가 교육에 미치는 영향'
🔍 'AI가 교육에 미치는 영향' 주제로 웹 검색 중...
# Report: AI가 교육에 미치는 영향

## 1. Keywords
AI 교육 영향, 인공지능 교육 적용 사례, AI 교육 연구, 인공지능 교육 혁신, AI 교육 도전과제, AI 교육 윤리

## 2. Sources
- https://www.seic.online/
- https://wizape.com/%ED%95%9C%EA%B5%AD%EC%96%B4/%ED%95%99%EC%83%9D-%ED%95%99%EC%8A%B5%EC%97%90-%EB%8C%80%ED%95%9C-AI-%EC%98%81%ED%96%A5-%ED%8F%89%EA%B0%80
- https://www.ibm.com/kr-ko/think/insights/impact-of-ai
- https://commbooks.com/%EB%8F%84%EC%84%9C/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EA%B3%BC-%EA%B3%A0%EC%A0%84-%EC%9D%BD%EA%B8%B0/

## 3. Summary
인공지능(AI)은 교육 분야에서 개인 맞춤형 학습, 교사의 업무 효율성 향상, 그리고 교육 콘텐츠의 혁신을 통해 긍정적인 변화를 이끌고 있습니다. 그러나 AI의 도입은 데이터 프라이버시, 윤리적 문제, 그리고 기술 격차와 같은 도전과제도 수반합니다. 이러한 문제를 해결하기 위해서는 신중한 정책 수립과 지속적인 연구가 필요합니다. AI의 교육 분야 활용은 향후 교육의 질과 접근성을 향상시키는 데 중요한 역할을 할 것으로 기대됩니다.

## 4. Body
## 서론

인공지능(AI)은 다양한 산업 분야에서 혁신을 주도하고 있으며, 교육 분야도 예외는 아닙니다. AI의 도입은 교육의 접근성과 효율성을 높이고, 개인화된 학습 경험을 제공하는 등 여러 긍정적인 변화를 가져오고 있습니다. 그러나 이러한 변화는 동시에 새로운 도전과제를 수반하며, 이에 대한 신중한 접근이 

In [5]:
# 4. 대화 흐름 제어 및 메인 챗봇
def add_to_history(role: str, content: str):
    """대화 히스토리에 메시지 추가"""
    global CONVERSATION_HISTORY
    CONVERSATION_HISTORY.append({"role": role, "content": content})
    
    # 히스토리 길이 제한
    if len(CONVERSATION_HISTORY) > MAX_HISTORY_LENGTH:
        CONVERSATION_HISTORY = CONVERSATION_HISTORY[-MAX_HISTORY_LENGTH:]

def process_user_input(user_input: str) -> str:
    """
    사용자 입력을 처리하는 메인 함수
    
    Args:
        user_input: 사용자가 입력한 텍스트
        
    Returns:
        str: 챗봇의 응답
    """
    # 종료 명령어 확인
    if user_input.lower() in ['quit', 'exit', '종료', 'q', '끝']:
        return "대화를 종료합니다. 감사합니다! 👋"
    
    # 주제 명확성 판단
    if is_topic_vague(user_input):
        # Clarify 단계
        clarify_attempts = 0
        while clarify_attempts < MAX_CLARIFY_ATTEMPTS:
            clarify_attempts += 1
            question = clarify_question(user_input, clarify_attempts)
            add_to_history("assistant", question)
            return question
        else:
            # 최대 시도 횟수 초과
            add_to_history("assistant", "주제를 좀 더 구체적으로 말씀해 주실 수 있을까요?")
            return "주제를 좀 더 구체적으로 말씀해 주실 수 있을까요?"
    else:
        # Research 단계 - 리포트 생성
        try:
            report = generate_report(user_input)
            formatted_report = format_report(report, user_input)
            add_to_history("assistant", formatted_report)
            return formatted_report
        except Exception as e:
            error_msg = f"리포트 생성 중 오류가 발생했습니다: {str(e)}"
            add_to_history("assistant", error_msg)
            return error_msg

def chat_loop():
    """
    메인 대화 루프
    """
    print("🤖 AI 리포트 자동화 챗봇에 오신 것을 환영합니다!")
    print("📝 궁금한 주제를 입력하시면 자동으로 리포트를 생성해드립니다.")
    print("💡 종료하려면 'quit', 'exit', '종료' 중 하나를 입력하세요.")
    print("=" * 60)
    
    # 초기화
    global CONVERSATION_HISTORY
    CONVERSATION_HISTORY = []
    
    while True:
        try:
            # 사용자 입력 받기
            user_input = input("\n👤 사용자: ").strip()
            
            if not user_input:
                print("❌ 입력이 비어있습니다. 다시 입력해주세요.")
                continue
            
            # 대화 히스토리에 사용자 입력 추가
            add_to_history("user", user_input)
            
            # 처리 및 응답
            response = process_user_input(user_input)
            
            # 응답 출력
            print(f"\n🤖 챗봇: {response}")
            
            # 종료 조건 확인
            if user_input.lower() in ['quit', 'exit', '종료', 'q', '끝']:
                break
                
        except KeyboardInterrupt:
            print("\n\n👋 대화를 종료합니다.")
            break
        except Exception as e:
            print(f"\n❌ 오류가 발생했습니다: {e}")
            print("다시 시도해주세요.")

# 대화형 테스트 (선택사항)
def test_chat():
    """간단한 테스트용 대화"""
    test_cases = [
        "AI",  # 모호한 주제
        "AI가 교육에 미치는 영향",  # 명확한 주제
        "경제",  # 모호한 주제
        "2024년 한국 부동산 시장 전망"  # 명확한 주제
    ]
    
    print("🧪 대화 테스트 시작:")
    print("=" * 40)
    
    for i, test_input in enumerate(test_cases, 1):
        print(f"\n[테스트 {i}] 입력: '{test_input}'")
        response = process_user_input(test_input)
        print(f"응답: {response[:100]}..." if len(response) > 100 else f"응답: {response}")
        print("-" * 40)

print("✅ 대화 흐름 제어 함수 구현 완료")
print("💡 chat_loop() 함수를 호출하여 대화를 시작할 수 있습니다.")
print("💡 test_chat() 함수로 간단한 테스트를 실행할 수 있습니다.")


✅ 대화 흐름 제어 함수 구현 완료
💡 chat_loop() 함수를 호출하여 대화를 시작할 수 있습니다.
💡 test_chat() 함수로 간단한 테스트를 실행할 수 있습니다.


In [6]:
# 5. 에러 처리 및 예외 상황 처리
def handle_api_error(error):
    error_msg = str(error).lower()
    
    if "rate limit" in error_msg or "quota" in error_msg:
        return "API 사용량이 초과되었습니다. 잠시 후 다시 시도해주세요."
    elif "authentication" in error_msg or "api key" in error_msg:
        return "API 인증에 문제가 있습니다. API 키를 확인해주세요."
    elif "network" in error_msg or "connection" in error_msg:
        return "네트워크 연결에 문제가 있습니다. 인터넷 연결을 확인해주세요."
    else:
        return f"시스템 오류가 발생했습니다: {str(error)}"

def validate_topic(topic):
    if not topic or not topic.strip():
        return False, "주제를 입력해주세요."
    
    if len(topic.strip()) < 2:
        return False, "주제가 너무 짧습니다. 더 구체적으로 입력해주세요."
    
    if len(topic) > 500:
        return False, "주제가 너무 깁니다. 500자 이내로 입력해주세요."
    
    return True, ""

def safe_generate_report(topic):
    is_valid, error_msg = validate_topic(topic)
    if not is_valid:
        return {
            "keywords": [topic],
            "sources": ["입력 오류"],
            "summary": error_msg,
            "body": error_msg
        }
    
    try:
        return generate_report(topic)
    except Exception as e:
        error_response = handle_api_error(e)
        return {
            "keywords": [topic],
            "sources": ["오류 발생"],
            "summary": error_response,
            "body": error_response
        }

def safe_is_topic_vague(topic):
    try:
        return is_topic_vague(topic)
    except Exception as e:
        print(f"⚠️ 주제 명확성 판단 중 오류: {e}")
        return True

def safe_clarify_question(topic, attempt=1):
    try:
        return clarify_question(topic, attempt)
    except Exception as e:
        print(f"⚠️ Clarify 질문 생성 중 오류: {e}")
        return "주제를 좀 더 구체적으로 말씀해 주실 수 있을까요?"

def process_user_input_safe(user_input):
    if user_input.lower() in ['quit', 'exit', '종료', 'q', '끝']:
        return "대화를 종료합니다. 감사합니다! 👋"
    
    is_valid, error_msg = validate_topic(user_input)
    if not is_valid:
        return f"❌ {error_msg}"
    
    if safe_is_topic_vague(user_input):
        clarify_attempts = 0
        while clarify_attempts < MAX_CLARIFY_ATTEMPTS:
            clarify_attempts += 1
            question = safe_clarify_question(user_input, clarify_attempts)
            add_to_history("assistant", question)
            return question
        else:
            add_to_history("assistant", "주제를 좀 더 구체적으로 말씀해 주실 수 있을까요?")
            return "주제를 좀 더 구체적으로 말씀해 주실 수 있을까요?"
    else:
        report = safe_generate_report(user_input)
        formatted_report = format_report(report, user_input)
        add_to_history("assistant", formatted_report)
        return formatted_report

def chat_loop_safe():
    print("🤖 AI 리포트 자동화 챗봇에 오신 것을 환영합니다!")
    print("📝 궁금한 주제를 입력하시면 자동으로 리포트를 생성해드립니다.")
    print("💡 종료하려면 'quit', 'exit', '종료' 중 하나를 입력하세요.")
    print("=" * 60)
    
    global CONVERSATION_HISTORY
    CONVERSATION_HISTORY = []
    
    while True:
        try:
            user_input = input("\n👤 사용자: ").strip()
            
            if not user_input:
                print("❌ 입력이 비어있습니다. 다시 입력해주세요.")
                continue
            
            add_to_history("user", user_input)
            response = process_user_input_safe(user_input)
            print(f"\n🤖 챗봇: {response}")
            
            if user_input.lower() in ['quit', 'exit', '종료', 'q', '끝']:
                break
                
        except KeyboardInterrupt:
            print("\n\n👋 대화를 종료합니다.")
            break
        except Exception as e:
            print(f"\n❌ 예상치 못한 오류가 발생했습니다: {e}")
            print("다시 시도해주세요.")

print("✅ 에러 처리 및 예외 상황 처리 구현 완료")
print("💡 chat_loop_safe() 함수를 사용하여 안전한 대화를 시작할 수 있습니다.")


✅ 에러 처리 및 예외 상황 처리 구현 완료
💡 chat_loop_safe() 함수를 사용하여 안전한 대화를 시작할 수 있습니다.


In [7]:
# 6. 테스트 케이스 및 검증
def run_comprehensive_test():
    """종합 테스트 실행"""
    print("🧪 AI 리포트 자동화 챗봇 종합 테스트 시작")
    print("=" * 60)
    
    # 테스트 케이스 정의
    test_cases = [
        {
            "category": "모호한 주제",
            "topics": ["AI", "경제", "환경", "기술"],
            "expected": "Clarify 질문 생성"
        },
        {
            "category": "명확한 주제", 
            "topics": [
                "AI가 교육에 미치는 영향",
                "2024년 한국 부동산 시장 전망",
                "코로나19가 온라인 쇼핑에 미친 영향",
                "블록체인 기술의 금융 분야 활용"
            ],
            "expected": "리포트 생성"
        },
        {
            "category": "경계선 케이스",
            "topics": ["", "a", "매우긴주제" * 100],
            "expected": "입력 검증 오류"
        }
    ]
    
    results = []
    
    for test_group in test_cases:
        print(f"\n📋 {test_group['category']} 테스트:")
        print("-" * 40)
        
        for topic in test_group["topics"]:
            print(f"\n🔍 테스트: '{topic[:50]}{'...' if len(topic) > 50 else ''}'")
            
            try:
                # 입력 검증 테스트
                is_valid, error_msg = validate_topic(topic)
                
                if not is_valid:
                    print(f"  ✅ 입력 검증 통과: {error_msg}")
                    results.append({"topic": topic, "status": "validation_error", "message": error_msg})
                    continue
                
                # 주제 명확성 판단 테스트
                is_vague = safe_is_topic_vague(topic)
                print(f"  📊 주제 명확성: {'모호함' if is_vague else '명확함'}")
                
                if is_vague:
                    # Clarify 테스트
                    question = safe_clarify_question(topic)
                    print(f"  ❓ Clarify 질문: {question[:100]}...")
                    results.append({"topic": topic, "status": "clarify", "message": question})
                else:
                    # 리포트 생성 테스트
                    print("  🔍 리포트 생성 중...")
                    report = safe_generate_report(topic)
                    
                    # 리포트 구조 검증
                    required_fields = ["keywords", "sources", "summary", "body"]
                    missing_fields = [field for field in required_fields if field not in report]
                    
                    if missing_fields:
                        print(f"  ❌ 리포트 구조 오류: {missing_fields} 필드 누락")
                        results.append({"topic": topic, "status": "report_error", "message": f"누락 필드: {missing_fields}"})
                    else:
                        print(f"  ✅ 리포트 생성 성공")
                        print(f"    - 키워드: {len(report['keywords'])}개")
                        print(f"    - 출처: {len(report['sources'])}개")
                        print(f"    - 요약 길이: {len(report['summary'])}자")
                        print(f"    - 본문 길이: {len(report['body'])}자")
                        results.append({"topic": topic, "status": "success", "message": "리포트 생성 성공"})
                
            except Exception as e:
                print(f"  ❌ 테스트 실패: {str(e)}")
                results.append({"topic": topic, "status": "error", "message": str(e)})
    
    # 테스트 결과 요약
    print("\n" + "=" * 60)
    print("📊 테스트 결과 요약")
    print("=" * 60)
    
    status_counts = {}
    for result in results:
        status = result["status"]
        status_counts[status] = status_counts.get(status, 0) + 1
    
    for status, count in status_counts.items():
        print(f"  {status}: {count}개")
    
    total_tests = len(results)
    success_rate = (status_counts.get("success", 0) + status_counts.get("clarify", 0)) / total_tests * 100
    print(f"\n🎯 전체 성공률: {success_rate:.1f}%")
    
    return results

def test_specific_scenarios():
    """특정 시나리오 테스트"""
    print("\n🎭 특정 시나리오 테스트")
    print("-" * 40)
    
    scenarios = [
        {
            "name": "연속 Clarify 테스트",
            "test": lambda: [safe_clarify_question("AI", i) for i in range(1, 4)]
        },
        {
            "name": "빈 입력 처리",
            "test": lambda: validate_topic("")
        },
        {
            "name": "매우 긴 입력 처리", 
            "test": lambda: validate_topic("a" * 1000)
        },
        {
            "name": "특수 문자 처리",
            "test": lambda: validate_topic("AI & Machine Learning @ 2024!")
        }
    ]
    
    for scenario in scenarios:
        print(f"\n🔬 {scenario['name']}:")
        try:
            result = scenario["test"]()
            print(f"  ✅ 성공: {result}")
        except Exception as e:
            print(f"  ❌ 실패: {e}")

# 테스트 실행
print("🧪 테스트 케이스 구현 완료")
print("💡 run_comprehensive_test() 함수를 호출하여 종합 테스트를 실행할 수 있습니다.")
print("💡 test_specific_scenarios() 함수로 특정 시나리오를 테스트할 수 있습니다.")


🧪 테스트 케이스 구현 완료
💡 run_comprehensive_test() 함수를 호출하여 종합 테스트를 실행할 수 있습니다.
💡 test_specific_scenarios() 함수로 특정 시나리오를 테스트할 수 있습니다.


In [8]:
# 7. 메인 실행 및 사용법 안내
def main():
    """메인 실행 함수"""
    print("🚀 AI 리포트 자동화 챗봇 시작")
    print("=" * 60)
    print("사용 가능한 명령어:")
    print("1. chat_loop_safe() - 안전한 대화 모드 시작")
    print("2. run_comprehensive_test() - 종합 테스트 실행")
    print("3. test_specific_scenarios() - 특정 시나리오 테스트")
    print("4. test_chat() - 간단한 대화 테스트")
    print("=" * 60)
    
    # API 키 확인
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY가 설정되지 않았습니다.")
        print("💡 .env 파일에 OPENAI_API_KEY=your_api_key를 추가하세요.")
        return
    
    print("✅ 모든 설정이 완료되었습니다!")
    print("💡 chat_loop_safe()를 호출하여 대화를 시작하세요.")

# 사용법 예시
def show_usage_examples():
    """사용법 예시 표시"""
    print("\n📖 사용법 예시:")
    print("-" * 40)
    
    examples = [
        {
            "input": "AI",
            "output": "AI에 대해 어떤 분야가 궁금하신가요? 교육, 의료, 자율주행, 창작 중에서 선택해주세요."
        },
        {
            "input": "AI가 교육에 미치는 영향",
            "output": "리포트 생성 (Keywords, Sources, Summary, Body 포함)"
        },
        {
            "input": "",
            "output": "❌ 주제를 입력해주세요."
        }
    ]
    
    for i, example in enumerate(examples, 1):
        print(f"\n예시 {i}:")
        print(f"  입력: '{example['input']}'")
        print(f"  출력: {example['output']}")

# 평가 기준 확인
def check_evaluation_criteria():
    """평가 기준 확인"""
    print("\n📋 평가 기준:")
    print("-" * 40)
    
    criteria = [
        "🎯 Design Requirements 충족: 명확한 Clarify/Research 분기 및 리포트 구성",
        "🧭 흐름 제어 정확도: 사용자 입력에 따른 자연스러운 진행",
        "🔍 신뢰성: 실제 검색 기반 리포트인지 여부",
        "🧩 코드 완성도: 기능적 일관성, 에러 처리 포함 여부",
        "💬 UX 자연스러움: 대화 맥락이 인간스럽고 매끄러운가"
    ]
    
    for criterion in criteria:
        print(f"  {criterion}")

# 실행
if __name__ == "__main__":
    main()
    show_usage_examples()
    check_evaluation_criteria()

print("✅ 모든 구현이 완료되었습니다!")
print("🎉 AI 리포트 자동화 챗봇이 준비되었습니다!")


🚀 AI 리포트 자동화 챗봇 시작
사용 가능한 명령어:
1. chat_loop_safe() - 안전한 대화 모드 시작
2. run_comprehensive_test() - 종합 테스트 실행
3. test_specific_scenarios() - 특정 시나리오 테스트
4. test_chat() - 간단한 대화 테스트
✅ 모든 설정이 완료되었습니다!
💡 chat_loop_safe()를 호출하여 대화를 시작하세요.

📖 사용법 예시:
----------------------------------------

예시 1:
  입력: 'AI'
  출력: AI에 대해 어떤 분야가 궁금하신가요? 교육, 의료, 자율주행, 창작 중에서 선택해주세요.

예시 2:
  입력: 'AI가 교육에 미치는 영향'
  출력: 리포트 생성 (Keywords, Sources, Summary, Body 포함)

예시 3:
  입력: ''
  출력: ❌ 주제를 입력해주세요.

📋 평가 기준:
----------------------------------------
  🎯 Design Requirements 충족: 명확한 Clarify/Research 분기 및 리포트 구성
  🧭 흐름 제어 정확도: 사용자 입력에 따른 자연스러운 진행
  🔍 신뢰성: 실제 검색 기반 리포트인지 여부
  🧩 코드 완성도: 기능적 일관성, 에러 처리 포함 여부
  💬 UX 자연스러움: 대화 맥락이 인간스럽고 매끄러운가
✅ 모든 구현이 완료되었습니다!
🎉 AI 리포트 자동화 챗봇이 준비되었습니다!


In [11]:
# 안전한 대화 모드 시작
chat_loop_safe()

🤖 AI 리포트 자동화 챗봇에 오신 것을 환영합니다!
📝 궁금한 주제를 입력하시면 자동으로 리포트를 생성해드립니다.
💡 종료하려면 'quit', 'exit', '종료' 중 하나를 입력하세요.

🤖 챗봇: 커피에 대해 어떤 부분이 궁금하신가요? 커피의 종류, 제조 과정, 또는 건강에 미치는 영향 중에서 선택해 주세요.

🤖 챗봇: ❌ 주제가 너무 짧습니다. 더 구체적으로 입력해주세요.

🤖 챗봇: 커피의 종류에 대해 어떤 부분이 궁금하신가요? 에스프레소, 라떼, 카푸치노의 차이점이나 원두의 종류에 대해 알고 싶으신가요?
🔍 '원두의 종류' 주제로 웹 검색 중...

🤖 챗봇: # Report: 원두의 종류

## 1. Keywords
원두 종류, 아라비카 원두, 로부스타 원두, 리베리카 원두, 엑셀사 원두, 커피 원산지별 특징, 커피 품종, 커피 가공 방식

## 2. Sources
- https://www.urbanbrush.net/downloads/%EC%9B%90%EB%91%90-%EC%A2%85%EB%A5%98-%EC%84%A4%EB%AA%85-pop-%EB%AC%B4%EB%A3%8C%EC%8B%9C%EC%95%88-type-beans-sample/
- https://with-hs.tistory.com/291
- https://warm002.tistory.com/101

## 3. Summary
커피 원두는 주로 아라비카, 로부스타, 리베리카, 엑셀사 네 가지 품종으로 분류됩니다. 아라비카는 고지대에서 재배되며 부드러운 맛과 향이 특징입니다. 로부스타는 저지대에서 자라며 강한 쓴맛과 높은 카페인 함량을 가집니다. 리베리카와 엑셀사는 생산량이 적어 덜 알려져 있지만, 각각 독특한 맛과 향을 제공합니다. 원산지와 가공 방식에 따라 원두의 맛과 향이 다양하게 나타납니다.

## 4. Body
커피는 전 세계적으로 사랑받는 음료로, 그 맛과 향은 사용되는 원두의 종류에 따라 크게 달라집니다. 원두는 주로 아라비카(Arabica), 로부스타(Robusta), 리베